<a href="https://colab.research.google.com/github/Sreekar-Kandhadai/pyspark-interview-questions/blob/main/cognizant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
𝐐𝐮𝐞𝐬𝐭𝐢𝐨𝐧:
You have the following dataset containing sales information for different products and
regions. Reshape the data using PySpark's pivot() method to calculate the total sales for
each product across regions, and then optimize it further by applying specific transformations.

Task 1: Use pivot() to create a table showing the total sales for each product by region.

Task 2: Add a column calculating the percentage contribution of each region to the total sales for that product.

Task 3: Sort the data in descending order by total sales for each product.

𝐬𝐜𝐡𝐞𝐦𝐚 𝐚𝐧𝐝 𝐝𝐚𝐭𝐚𝐬𝐞𝐭
data = [ ("North", "Laptop", 2000, "Q1"), ("South", "Laptop", 3000, "Q1"), ("East", "Laptop", 2500, "Q1"), ("North", "Phone", 1500, "Q1"),
("South", "Phone", 1000, "Q1"), ("East", "Phone", 2000, "Q1"),
("North", "Laptop", 3000, "Q2"), ("South", "Laptop", 4000, "Q2"),
("East", "Laptop", 3500, "Q2"), ("North", "Phone", 2500, "Q2"),
("South", "Phone", 1500, "Q2"), ("East", "Phone", 3000, "Q2"), ]

columns = ["Region", "Product", "Sales", "Quarter"]


In [10]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *

spark=SparkSession.builder.appName("learning").getOrCreate()

data=[ ("North", "Laptop", 2000, "Q1"), ("South", "Laptop", 3000, "Q1"), ("East", "Laptop", 2500, "Q1"), ("North", "Phone", 1500, "Q1"),
("South", "Phone", 1000, "Q1"), ("East", "Phone", 2000, "Q1"),
("North", "Laptop", 3000, "Q2"), ("South", "Laptop", 4000, "Q2"),
("East", "Laptop", 3500, "Q2"), ("North", "Phone", 2500, "Q2"),
("South", "Phone", 1500, "Q2"), ("East", "Phone", 3000, "Q2"), ]

schema=["Region", "Product", "Sales", "Quarter"]

df=spark.createDataFrame(data,schema)

df.show()

df1=df.groupBy('product').pivot('region').sum('sales')

df1.show()

df2=df1.withColumn('totalsales',col('East')+col('North')+col('South')).\
    withColumn('North_%',round(col('North')/col('totalsales')*100,2))\
    .withColumn('South_%',round(col('South')/col('totalsales')*100,2)).\
    withColumn('East_%',round(col('East')/col('totalsales')*100,2)).orderBy(col('totalsales').desc())

df2.show()



+------+-------+-----+-------+
|Region|Product|Sales|Quarter|
+------+-------+-----+-------+
| North| Laptop| 2000|     Q1|
| South| Laptop| 3000|     Q1|
|  East| Laptop| 2500|     Q1|
| North|  Phone| 1500|     Q1|
| South|  Phone| 1000|     Q1|
|  East|  Phone| 2000|     Q1|
| North| Laptop| 3000|     Q2|
| South| Laptop| 4000|     Q2|
|  East| Laptop| 3500|     Q2|
| North|  Phone| 2500|     Q2|
| South|  Phone| 1500|     Q2|
|  East|  Phone| 3000|     Q2|
+------+-------+-----+-------+

+-------+----+-----+-----+
|product|East|North|South|
+-------+----+-----+-----+
|  Phone|5000| 4000| 2500|
| Laptop|6000| 5000| 7000|
+-------+----+-----+-----+

+-------+----+-----+-----+----------+-------+-------+------+
|product|East|North|South|totalsales|North_%|South_%|East_%|
+-------+----+-----+-----+----------+-------+-------+------+
| Laptop|6000| 5000| 7000|     18000|  27.78|  38.89| 33.33|
|  Phone|5000| 4000| 2500|     11500|  34.78|  21.74| 43.48|
+-------+----+-----+-----+---------